In [1]:
import os, sys, random, time, pickle
src_dir_ = '/home/tan/Documents/GitHub/pdpt_2022/src'
sys.path.insert(1, src_dir_)

import numpy as np
from pathlib import Path

from util import read_pickle, group_cycle_truck, manual_stop
from pdpt_route_schedule import gurobi_master_cycle, greedy_fix_MP, MP_to_SP, cpo_sub, greedy_heuristic, time_checker_cluster, calculate_SP_cost, postprocess_pdpt_solution

dir_ = '/home/tan/Documents/GitHub/pdpt_2022/'
num_ins = 10


In [2]:
def pdpt_route_schedule_decomposition(path_, ins, subroutes, verbose):
    """
    The main function of PDPT decomposition
    input: ct = count
    
    output:
        infeasible_clusters
        removed_cargo_MP
        removed_cargo_SP

    Three steps to handle removed cargos:
        1. add a removed cargo to a different cluster, resolve the MP-SP
        2. add all removed cargos to an unused truck
        3. LNS
    """    
    

    constant = ins['constant']

    # single_truck_deviation = ins['single_truck_deviation']
    if subroutes is None:
        selected_truck = ins['truck']
        selected_cargo = ins['cargo']
        selected_node = ins['nodes']
        selected_edge = ins['edge_shortest']
    else:
        selected_cargo, selected_truck, selected_node, selected_edge = subroutes

    # Setting parameters for clustering cargo and truck 
    runtime = 100
    # count_node_cluster = 10
    # conflict_obj_coefficient = 10000
    
      
    ###### Solve the MP+SP for each cluster ######
    
    time_start = time.time()
    
    removed_cargo = {}
    # solution_cluster = {}   


    ###### Initialize data structures for the solution of the current cluster ######

    # solution_created_truck = {}

    ### group cycle and non-cycle trucks
    created_truck_yCycle, created_truck_nCycle, created_truck_all = group_cycle_truck(selected_truck) 


    ###### Master Problem Solved by Gurobi MIP ######

    # no variable ordering
    # heuristic = 0.2

    #x_sol: x^k_{ij}, if truck k visit edge (i,j) or not
    #s_sol: s^k, if truck k i used or not
    #z_sol: z^{kr}_{ij}, if truck k carries parcel r visit edge (i, j) or not
    #y_sol: y^k_r, if parcel r is carried by truck k
    #u_sol: y^r_i, if parcel r is transfered at node i
    #D_sol: D^k_i, depature time of truck k at node i

    gurobi_log_file = os.path.join(path_, 'routingMP.log')
    
    obj_val_MP, runtime_MP, \
    x_sol, s_sol, z_sol, y_sol, u_sol, D_sol, Db_sol, \
    cost_truck_value, cost_travel_value, cost_transfer_value = \
    gurobi_master_cycle(constant, selected_cargo, 
        created_truck_yCycle, created_truck_nCycle, created_truck_all, 
        selected_edge, selected_node, runtime*1, gurobi_log_file, verbose)


    if obj_val_MP >= 0:
        print(f'MP feasible, move to SP')
    elif obj_val_MP < 0:
        print( '+++ MP infeasible')

        return False, None
        # # print(f'MP infeasible, stop solving, skip to the next iter of RASD')
        # # return False, {}
        # selected_cargo, created_truck_all, selected_edge, selected_node, \
        # selected_removed_cargo, obj_val_MP, runtime_MP, \
        # x_sol, s_sol, z_sol, y_sol, u_sol, D_sol = \
        # greedy_fix_MP(constant, selected_cargo, 
        #             created_truck_yCycle, created_truck_nCycle, created_truck_all, 
        #             selected_edge, selected_node, runtime, gurobi_log_file, verbose)
        # # find removed cargo from MP
        # for c in selected_removed_cargo:
        #     removed_cargo[c] = selected_cargo[c]

        

    
    ###### The MP should be feasible here ######

    ###### Subproblem Solved by CPO CP ######

    # convert MP solution to CP parameters
    print(f'Convert MP solution to SP parameters')

    truck_MP, truck_nodes, truck_nodes_index, cargo_in, cargo_out, \
    transfer_nodes, cargo_unload, cargo_load = \
    MP_to_SP(constant, selected_cargo, 
        created_truck_yCycle, created_truck_nCycle, created_truck_all, 
        selected_edge, selected_node,
        x_sol, s_sol, z_sol, y_sol, u_sol, D_sol)  
    

    # Evaluating the feasibility of SP
    cplex_log_file = os.path.join(path_, 'schedulingSP.log')

    feasibility_SP, g_sol, h_sol, D_sol = \
    cpo_sub(constant, selected_cargo, 
        created_truck_yCycle, created_truck_nCycle, created_truck_all, 
        selected_edge, selected_node,
        truck_MP, truck_nodes, truck_nodes_index, 
        cargo_in, cargo_out,
        transfer_nodes, cargo_unload, cargo_load,
        runtime, cplex_log_file)

    if feasibility_SP != 'Feasible':

        print( '+++ SP infeasible')
        return False, None
        # print(f'SP infeasible, repair with greedy heuristic')

        # # Greedy heuristic for cargo removal
        # selected_removed_cargo, \
        # truck_MP, truck_nodes, truck_nodes_index, \
        # cargo_in, cargo_out, \
        # transfer_nodes, cargo_unload, cargo_load = \
        # greedy_heuristic(constant, selected_cargo, 
        #     created_truck_yCycle, created_truck_nCycle, created_truck_all, 
        #     selected_edge, selected_node,
        #     truck_MP, truck_nodes, truck_nodes_index,
        #     cargo_in, cargo_out,
        #     transfer_nodes, cargo_unload, cargo_load, cplex_log_file, verbose)

        # # find removed cargo from SP
        # for c in selected_removed_cargo:
        #     removed_cargo[c] = selected_cargo[c]
    else:
        print(f'SP Feasible, Check time constraint')
        if time_checker_cluster(constant, selected_cargo, created_truck_all, 
                            selected_edge, truck_MP, truck_nodes, 
                            cargo_unload, cargo_load, g_sol, h_sol, D_sol):
            print('Passing time constraint check!')
        else:
            print('Failed time constraint check, exit!!!!')
            print(f'Failed time constraint check, skip to the next iter of RASD')
            return False, None
            # sys.exit()
        
    # Calculate SP costs: truck cost + traveling cost + transfer cost
    if verbose > 0:
        print(f'Compute cost after solving SP')

    truck_cost, travel_cost, transfer_cost = \
    calculate_SP_cost(constant, selected_cargo, selected_edge, 
        truck_MP, truck_nodes, 
        cargo_in, cargo_out, transfer_nodes)


    truck_used, cargo_delivered, cargo_undelivered, \
    trucks_per_cargo, cargo_in_truck, truck_route, cargo_route =  postprocess_pdpt_solution(subroutes, x_sol, s_sol, y_sol, z_sol, u_sol, verbose = 0)

    print(f'===== summary of post-processing [{selected_truck.keys()}] route+schedule solution')
    print(f'+++ cargo to truck assignment')
    for key, value in cargo_in_truck.items():
        print(f'      {key}: {value}')
    print(f'+++ undelivered cargos:\n     {removed_cargo}')

            
    if verbose >0:
        print(f'+++ truck to cargo assignment')
        for key, value in trucks_per_cargo.items():
            print(f'      {key}: {value}')
        print(f'+++ truck route')
        for key, value in truck_route.items():
            print(f'      {key}: {value}')
        print(f'+++ cargo route')
        for key, value in cargo_route.items():
            print(f'      {key}: {value}')

    res = {'MP': {'x_sol': x_sol,
                  's_sol': s_sol,
                  'z_sol': z_sol,
                  'y_sol': y_sol,
                  'u_sol': u_sol,
                  'D_sol': D_sol,
                  'Db_sol': Db_sol,
                 },
            'SP':{'g_sol': g_sol,
                  'h_sol': h_sol,
                  'D_sol': D_sol,
                 },
            'route':{'removed_cargo':removed_cargo,
                     'used_truck':truck_used,
                     'truck_route': truck_route,
                     'cargo_route': cargo_route,
                     'transfer_nodes': transfer_nodes,
                    },
            'cost':{'truck_cost': truck_cost,
                    'travel_cost': travel_cost,
                    'transfer_cost': transfer_cost,
                    }
            }

    return True, res

In [3]:
for case_num in range(1,6,1):
    for num_trucks in [2, 3]:
        for i in range(num_ins):
            pdpt_ins_filename = os.path.join(dir_, f'data/case{case_num}', f'case{case_num}_truck{num_trucks}_ins{i+1}.pkl')
            print(f'===== START route+schedule\n      {pdpt_ins_filename}')

            pdpt_ins = read_pickle(pdpt_ins_filename)

            assert len(pdpt_ins['truck'].keys()) == num_trucks
            n_cargo, n_node = len(pdpt_ins['cargo'].keys()), len(pdpt_ins['nodes']) 

            print(f'      {num_trucks} trucks, { n_cargo} cargos, { n_node} nodes ')
            path_ = os.path.join(dir_, f'out/pdpt', f'case{case_num}_truck{num_trucks}_ins{i+1}')


            Path(path_).mkdir(parents=True, exist_ok=True)
            start_runtime = time.time()
            MP_status, res = pdpt_route_schedule_decomposition(path_, pdpt_ins, None, 0)
            runtime = time.time() - start_runtime
            if MP_status == True:
                n_removed_cargos = res['route']['removed_cargo']
                print(f'===== END route+schedule after {runtime}s, found a solution with {n_removed_cargos} removed cargos')
                assert res is not None
                res['runtime'] = runtime
                pdpt_sol_filename = os.path.join(path_, f'case{case_num}_truck{num_trucks}_ins{i+1}_soln.pkl')
                with open(pdpt_sol_filename, 'wb') as pickle_file:
                    pickle.dump(res, pickle_file)
            else:
                print(f'===== END route+schedule after {runtime}s, route+schedule infeasible')


===== START route+schedule
      /home/tan/Documents/GitHub/pdpt_2022/data/case1/case1_truck2_ins1.pkl
      2 trucks, 16 cargos, 10 nodes 
Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-15
Set parameter TimeLimit to value 100
Set parameter LogFile to value "/home/tan/Documents/GitHub/pdpt_2022/out/pdpt/case1_truck2_ins1/routingMP.log"
Set parameter Heuristics to value 0.2
*********************** MP Feasible **********************
The Gurobi obj value is 105941
The Gurobi runtime is 33.772028

cost_truck_value: 60000.0
cost_travel_value: 45250.0
cost_transfer_value: 691.0
MP feasible, move to SP
Convert MP solution to SP parameters


******************** SP Infeasible **********************

+++ SP infeasible
===== END route+schedule after 33.991488456726074s, route+schedule infeasible
===== START route+schedule
      /home/tan/Documents/GitHub/pdpt_2022/data/case1/case1_truck2_ins2.pkl
      2 trucks, 42 cargos, 11 nodes 
Set parameter TimeLimi

TypeError: cannot unpack non-iterable NoneType object

<h3>Observations: the route+schedule decomposition is not working, even with small instance generated from PDOTW solution, the routing MP returns to infeasible.</h3>